In [1]:
!pip install elasticsearch
!pip install pandas
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import json
from kafka.consumer import KafkaConsumer
from elasticsearch import Elasticsearch
from elasticsearch import helpers


In [6]:
# Configuração do consumidor
bootstrap_servers = ['172.25.0.12:9092']
topic = 'yfinance-data'
consumer = KafkaConsumer(topic, bootstrap_servers=bootstrap_servers,
                         value_deserializer=lambda x: json.loads(x.decode('utf-8')))
# for message in consumer:
#     data = message.value
#     # Processar os dados
#     print(data['Open'])

In [8]:
# Criar um DataFrame para armazenar os dados
df = pd.DataFrame()

for message in consumer:
    data = message.value
    print(data)
    df = df.append(data, ignore_index=True)

# Analisar o DataFrame
print(df.describe())

KeyboardInterrupt: 

In [ ]:
# Configuração do Elasticsearch
es = Elasticsearch(['http://172.25.0.25:9200'])
index_name = 'yfinance_data'

# Criar o índice (se não existir)
es.indices.create(index=index_name, ignore=400)

In [ ]:
# Inserir os dados no Elasticsearch
for index, row in df.iterrows():
    es.index(index=index_name, id=index, body=row.to_dict())

In [ ]:
actions = []
for index, row in df.iterrows():
    actions.append({
        '_index': index_name,
        '_id': index,
        '_source': row.to_dict()
    })

# Executar a operação em bulk
helpers.bulk(es, actions)